## AI summarization of passenger chats

This notebook has AI summarization of baly passenger chats using llama3 (ollama APIs) and openAI.
The data is downloaded using freshchat rest apis. 

### Setup

You need 
  1. Input data frame, which has the freshchat data for any given period.
  2. openai .env file, which contains api key etc
  3. ollama endpoint

In [2]:
%%capture
pip install ollama openai

## Data Load
We first load the freshchat data, and then clean it up, to ensure we only have unique conversations, and all boilerplate text is removed.

In [43]:
import pandas as pd

df = pd.read_csv('data.passenger.20240524.csv', dtype = { 'conversation_id': str, 'user_name': str, 'messages': str},usecols = { 'user_name', 'conversation_id', 'messages' })
df.head()

,user_name,conversation_id,messages
0,علي بازوكه,28516b4d-3cef-4270-95b0-43ae1a3139c2,['Conversation reopened due to new incoming me...
1,هشام,180bba97-f80d-4cb4-b29c-e789620ae9f7,"['🖕🖕🖕', 'Conversation was reopened by هشام', '..."
2,علي الموسوي الموسوي,8e66eba4-46e8-42ce-9ee5-7d092c7f2af9,['Conversation was reopened by علي الموسوي ال...
3,همام علي,f9262057-afbc-4527-9b0a-cafb1e0e97d8,['إذا ممكن اني طلبت رحله قبل شهر تقريبا \nوالو...
4,عصام ابو عسل,58ea8ba5-c0e8-45b4-a72d-81fd61fe2c15,"['Conversation was reopened by عصام ابو عسل', ..."


In [73]:
df = df.drop_duplicates(subset='conversation_id', keep='first')
df.describe()

,user_name,conversation_id,messages
count,681,681,681
unique,651,681,662
top,علي,28516b4d-3cef-4270-95b0-43ae1a3139c2,['السلام عليكم']
freq,4,1,6


In [74]:
import re
removeChatRegex = r"^(Conversation|التحدث مع موظف الدعم|السلام عليكم).*"

df.head()

,user_name,conversation_id,messages
0,علي بازوكه,28516b4d-3cef-4270-95b0-43ae1a3139c2,['Conversation reopened due to new incoming me...
1,هشام,180bba97-f80d-4cb4-b29c-e789620ae9f7,"['🖕🖕🖕', 'Conversation was reopened by هشام', '..."
2,علي الموسوي الموسوي,8e66eba4-46e8-42ce-9ee5-7d092c7f2af9,['Conversation was reopened by علي الموسوي ال...
3,همام علي,f9262057-afbc-4527-9b0a-cafb1e0e97d8,['إذا ممكن اني طلبت رحله قبل شهر تقريبا \nوالو...
4,عصام ابو عسل,58ea8ba5-c0e8-45b4-a72d-81fd61fe2c15,"['Conversation was reopened by عصام ابو عسل', ..."


Seems like the following is a must do for reading array of strings from the file

In [75]:
from ast import literal_eval
df['messages'] = df['messages'].apply(literal_eval)

In [76]:
print(df['messages'][4])

['Conversation was reopened by عصام ابو عسل', 'التحدث مع موظف الدعم', 'سلام عليكم']


In [101]:
import re

def filter_common_strings(array):
    filtered_strings = [ 'Conversation reopened due to new incoming message', 'سلام عليكم']
    filtered_re = r"^(Conversation|BLY-).*"
    return [x for x in array if x not in filtered_strings and re.search(filtered_re,x) == None]

df['messages'] = df['messages'].apply(filter_common_strings)

In [102]:
df.head()

,user_name,conversation_id,messages
0,علي بازوكه,28516b4d-3cef-4270-95b0-43ae1a3139c2,"[نسيت سماعات, نسيت يمه سماعات ودك علي مي جاوب ..."
1,هشام,180bba97-f80d-4cb4-b29c-e789620ae9f7,"[🖕🖕🖕, اني دافحص البرنامج همه مرضه يوافقون راسا..."
2,علي الموسوي الموسوي,8e66eba4-46e8-42ce-9ee5-7d092c7f2af9,"[ان شاء الـلَّــــﷻـــه, يرجى الاهتمام بهذه ال..."
3,همام علي,f9262057-afbc-4527-9b0a-cafb1e0e97d8,[إذا ممكن اني طلبت رحله قبل شهر تقريبا \nوالول...
4,عصام ابو عسل,58ea8ba5-c0e8-45b4-a72d-81fd61fe2c15,[التحدث مع موظف الدعم]


## OpenAI Summarization

In [103]:
%%capture
pip install openai

In [104]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
load_dotenv()

True

In [105]:
aclient = AzureOpenAI(
        api_key=os.environ['AZURE_OPENAI_KEY'],
        api_version = "2023-05-15")

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
ENCODING_MODEL = "gpt-3.5-turbo"

In [106]:
import tiktoken
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

context = ''
token_length = 0

for x in df['messages']:
    chat = '.'.join(x)
    token_length += len(tokenizer.encode(chat))
    if token_length >= 3000:
        break
    context += '\n\n' + chat

print(token_length,context)

3171 

نسيت سماعات.نسيت يمه سماعات ودك علي مي جاوب .نسيت أغراضي خلال الرحلة

🖕🖕🖕.اني دافحص البرنامج همه مرضه يوافقون راسا حته ما الحك الغي .همه الكباتن ليش يوافقون على رحله نقطة الوصول والانطلاق نفس المكان .دفتح الحظر بابه

ان شاء الـلَّــــﷻـــه.يرجى الاهتمام بهذه الامور.فضلا عن نظافة وترتيب السيارة.وكذلك رقم ونوع ولون السيارة.غالبا الكابتن الموجود بالتطبيق يختلف عن الحقيقة

إذا ممكن اني طلبت رحله قبل شهر تقريبا 
والولد بقى يراسلني ويعاكسني وكتله اني مره مزوجه تلافي للمشكله 
ورجع كل شويه يراسلني اني. معجب بيج 
ومااذكر يارحله منهم بالضبط
بس ارقام ليراسلني منهم عندي.التحدث مع موظف الدعم.رجاءاً كلش محتاجتكم

التحدث مع موظف الدعم

استفسار عن المحفظة داخل التطبيق.تمام حبيبي .بعد الف كال ماعدي راح اضيفه عل محفضه .حبيبي اطيته خمسه رجعلي الف 

اي نعم نفس الشي صار بالرحلة الثانية

شكراً استاذ.اقتراحي تخفيض الاجور هذا يعود بمردود مادي قوي للشركة ومنافسه مشروعه وعدم قبول طلبات السواق وقتراح اخر تقديم جائزة افضل سائق شهرياً من خلال النقاط والتقييم الممتاز الي يحصل عليه هذا يشجعه على العمل بصورك م

In [108]:
def get_completion(prompt):
    messages = [
                {"role": "system", "content": "You are an expert analyst hired to help a ride hailing company improve their CSAT score.."},
                {"role":"user", "content":prompt}
               ]
    response = aclient.chat.completions.create(
            model = deployment,
            messages = messages,
            temperature = 0)
   #         max_tokens = 900)
    return response.choices[0].message.content

In [109]:
prompt = f"""
        Each line representas a conversation, containing messages from a single user in a session. Generate top 5 actionable insights for management. 
        For each of your insights, you must include conversations with original text and its english translation that best support it..
        ```{context}```
         """
response = get_completion(prompt)
print(response)

Insight 1: Improve driver behavior and professionalism
- Conversation: إذا ممكن اني طلبت رحله قبل شهر تقريبا والولد بقى يراسلني ويعاكسني وكتله اني مره مزوجه تلافي للمشكله ورجع كل شويه يراسلني اني. معجب بيج ومااذكر يارحله منهم بالضبط بس ارقام ليراسلني منهم عندي.التحدث مع موظف الدعم.رجاءاً كلش محتاجتكم
- Translation: If possible, I requested a ride about a month ago and the driver kept messaging me and harassing me, saying that I'm married to avoid the problem. He keeps messaging me every now and then, saying that I like him. I don't even remember the exact ride, but I have the numbers to contact them. I need to speak to a support representative. Please, I really need your help.

Insight 2: Address issues with payment and refunds
- Conversation: تمام شكراً جزيلاً ❤️.شكراً جزيلاً ويرجى اعلامنا بالتفاصيل للتأكد انو تم محاسبتة حتى نعرف الشركة مع الزبون وتحترم الزبون وانتم شركة رصينه ويجب ان تحفظون كرامة الزبون الي يتعامل وياكم لانو هذا رزقكم
- Translation: Thank you very much ❤️. Thank you 